In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go

df = pd.read_csv('../data/bike_data_cleaned_features.csv', index_col='id')

df.head()

,date,season,year,month,hour,is_holiday,weekday,is_workingday,weather_condition,temp_celsius,...,Rolling_Avg_Rent_24_h,Rolling_Avg_Rent_168_h,hourly_rental_deviation,hourly_rental_deviation_label,Lagged_Rent_1h,count_prev_week_same_hour,log_total_bike_ct,temp_type_diff,wind_cooling_effect,extreme_weather
id,,,,,,,,,,,,,,,,,,,,,
1,2011-01-01,1,0,1,0,0,6,0,1,0.24,...,16.000000,16.000000,-20.222222,below_average,0.0,0.0,2.833213,-0.0479,13.26916,0
2,2011-01-01,1,0,1,1,0,6,0,1,0.22,...,28.000000,28.000000,6.555556,above_average,16.0,0.0,3.713572,-0.0527,13.25673,0
3,2011-01-01,1,0,1,2,0,6,0,1,0.22,...,29.333333,29.333333,6.555556,above_average,40.0,0.0,3.496508,-0.0527,13.25673,0
4,2011-01-01,1,0,1,3,0,6,0,1,0.24,...,25.250000,25.250000,0.444444,above_average,32.0,0.0,2.639057,-0.0479,13.26916,0
5,2011-01-01,1,0,1,4,0,6,0,1,0.24,...,20.400000,20.400000,-2.888889,below_average,13.0,0.0,0.693147,-0.0479,13.26916,0


In [2]:
df.dtypes

date                              object
season                             int64
year                               int64
month                              int64
hour                               int64
is_holiday                         int64
weekday                            int64
is_workingday                      int64
weather_condition                  int64
temp_celsius                     float64
atem_celsius                     float64
humidity                         float64
windspeed                        float64
casual_user_ct                     int64
registered_user_ct                 int64
total_bike_ct                      int64
year_month                        object
day_part                          object
rush_hour                          int64
casual_to_registered_ratio       float64
Rolling_Avg_Rent_2_h             float64
Rolling_Avg_Rent_3_h             float64
Rolling_Avg_Rent_4_h             float64
Rolling_Avg_Rent_5_h             float64
Rolling_Avg_Rent

In [3]:
#Some Preprocessing
cat_var = df.select_dtypes(include=['object']).columns

for col in cat_var:
    if col != 'date':
        print(col, df[col].unique())

df.drop(columns=['year_month'], inplace=True)
# encoded to use 0 and 1 for binary variables
df['hourly_rental_deviation_label'] = df['hourly_rental_deviation_label'].map({'below_average': 0, 'above_average': 1})
df.drop(columns=['hourly_rental_deviation'], inplace=True)

# use pd.get_dummies for day_part
day_part_dummies = pd.get_dummies(df['day_part'], prefix='day_part')
df = pd.concat([df, day_part_dummies], axis=1)
df.drop(columns=['day_part'], inplace=True)

year_month ['2011-01-01' '2011-02-01' '2011-03-01' '2011-04-01' '2011-05-01'
 '2011-06-01' '2011-07-01' '2011-08-01' '2011-09-01' '2011-10-01'
 '2011-11-01' '2011-12-01' '2012-01-01' '2012-02-01' '2012-03-01'
 '2012-04-01' '2012-05-01' '2012-06-01' '2012-07-01' '2012-08-01'
 '2012-09-01' '2012-10-01' '2012-11-01' '2012-12-01']
day_part ['night' 'morning' 'afternoon' 'evening']
hourly_rental_deviation_label ['below_average' 'above_average']


In [4]:
#set date as index
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.head()

,season,year,month,hour,is_holiday,weekday,is_workingday,weather_condition,temp_celsius,atem_celsius,...,Lagged_Rent_1h,count_prev_week_same_hour,log_total_bike_ct,temp_type_diff,wind_cooling_effect,extreme_weather,day_part_afternoon,day_part_evening,day_part_morning,day_part_night
date,,,,,,,,,,,,,,,,,,,,,
2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,...,0.0,0.0,2.833213,-0.0479,13.26916,0,False,False,False,True
2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,...,16.0,0.0,3.713572,-0.0527,13.25673,0,False,False,False,True
2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,...,40.0,0.0,3.496508,-0.0527,13.25673,0,False,False,False,True
2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,...,32.0,0.0,2.639057,-0.0479,13.26916,0,False,False,False,True
2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,...,13.0,0.0,0.693147,-0.0479,13.26916,0,False,False,False,True


In [5]:
df['day_part_afternoon'] = df['day_part_afternoon'] * 1
df['day_part_evening'] = df['day_part_evening'] * 1
df['day_part_morning'] = df['day_part_morning'] * 1
df['day_part_night'] = df['day_part_night'] * 1

df.head()

,season,year,month,hour,is_holiday,weekday,is_workingday,weather_condition,temp_celsius,atem_celsius,...,Lagged_Rent_1h,count_prev_week_same_hour,log_total_bike_ct,temp_type_diff,wind_cooling_effect,extreme_weather,day_part_afternoon,day_part_evening,day_part_morning,day_part_night
date,,,,,,,,,,,,,,,,,,,,,
2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,...,0.0,0.0,2.833213,-0.0479,13.26916,0,0,0,0,1
2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,...,16.0,0.0,3.713572,-0.0527,13.25673,0,0,0,0,1
2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,...,40.0,0.0,3.496508,-0.0527,13.25673,0,0,0,0,1
2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,...,32.0,0.0,2.639057,-0.0479,13.26916,0,0,0,0,1
2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,...,13.0,0.0,0.693147,-0.0479,13.26916,0,0,0,0,1


In [6]:
df.dtypes

season                             int64
year                               int64
month                              int64
hour                               int64
is_holiday                         int64
weekday                            int64
is_workingday                      int64
weather_condition                  int64
temp_celsius                     float64
atem_celsius                     float64
humidity                         float64
windspeed                        float64
casual_user_ct                     int64
registered_user_ct                 int64
total_bike_ct                      int64
rush_hour                          int64
casual_to_registered_ratio       float64
Rolling_Avg_Rent_2_h             float64
Rolling_Avg_Rent_3_h             float64
Rolling_Avg_Rent_4_h             float64
Rolling_Avg_Rent_5_h             float64
Rolling_Avg_Rent_6_h             float64
Rolling_Avg_Rent_12_h            float64
Rolling_Avg_Rent_24_h            float64
Rolling_Avg_Rent

In [7]:
correlations = df.corr()

correlations

,season,year,month,hour,is_holiday,weekday,is_workingday,weather_condition,temp_celsius,atem_celsius,...,Lagged_Rent_1h,count_prev_week_same_hour,log_total_bike_ct,temp_type_diff,wind_cooling_effect,extreme_weather,day_part_afternoon,day_part_evening,day_part_morning,day_part_night
season,1.000000,-0.010742,0.830386,-0.006117,-0.009585,-0.002335,0.013743,-0.014524,0.312025,0.319380,...,0.178184,0.197379,0.167198,0.147308,0.123814,NaN,-0.003015,-0.003135,-0.001096,0.007292
year,-0.010742,1.000000,-0.010473,-0.003867,0.006692,-0.004485,-0.002196,-0.019157,0.040913,0.039222,...,0.250496,0.254451,0.168902,0.032254,-0.001799,NaN,-0.002439,-0.001642,-0.000568,0.004678
month,0.830386,-0.010473,1.000000,-0.005772,0.018430,0.010400,-0.003477,0.005400,0.201691,0.208096,...,0.120759,0.166997,0.114360,0.087178,0.114143,NaN,-0.002821,-0.003050,-0.000360,0.006271
hour,-0.006117,-0.003867,-0.005772,1.000000,0.000479,-0.003498,0.002285,-0.020203,0.137603,0.133750,...,0.474776,0.390747,0.564470,0.099533,-0.096304,NaN,0.163955,0.666590,-0.337646,-0.496610
is_holiday,-0.009585,0.006692,0.018430,0.000479,1.000000,-0.102088,-0.252471,-0.017036,-0.027340,-0.030973,...,-0.030182,-0.015374,-0.026007,0.001647,0.002770,NaN,0.000171,0.000217,0.000675,-0.001069
weekday,-0.002335,-0.004485,0.010400,-0.003498,-0.102088,1.000000,0.035955,0.003311,-0.001795,-0.008821,...,0.023673,0.027113,0.029496,0.033171,-0.004730,NaN,-0.001197,-0.002056,0.000793,0.002477
is_workingday,0.013743,-0.002196,-0.003477,0.002285,-0.252471,0.035955,1.000000,0.044672,0.055390,0.054667,...,0.026270,0.020603,0.004339,0.036032,0.015111,NaN,0.000326,0.001571,0.001474,-0.003391
weather_condition,-0.014524,-0.019157,0.005400,-0.020203,-0.017036,0.003311,0.044672,1.000000,-0.102640,-0.105563,...,-0.132447,-0.018568,-0.121874,-0.046002,-0.040332,NaN,0.010093,-0.033293,0.034700,-0.011523
temp_celsius,0.312025,0.040913,0.201691,0.137603,-0.027340,-0.001795,0.055390,-0.102640,1.000000,0.987672,...,0.406083,0.361036,0.388694,0.646969,0.049333,NaN,0.167843,0.063470,-0.120830,-0.111404
atem_celsius,0.319380,0.039222,0.208096,0.133750,-0.030973,-0.008821,0.054667,-0.105563,0.987672,1.000000,...,0.403962,0.355801,0.385523,0.519631,0.081174,NaN,0.160013,0.062456,-0.117205,-0.106145


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
['total_bike_ct','log_total_bike_ct', 'registered_user_ct', 'Rolling_Avg_Rent_2_h','Lagged_Rent_1h', 'casual_user_ct']
X = df.drop(columns=)
y = df['total_bike_ct']


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# rescaling the data with robuse scaler
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf_model = RandomForestRegressor(n_estimators=500, random_state=42)
rf_model.fit(X_train_scaled, y_train)

RandomForestRegressor(n_estimators=500, random_state=42)

In [10]:
df_importances = pd.DataFrame({'feature': X.columns, 'importance': rf_model.feature_importances_})

df_importances.sort_values(by='importance', ascending=False, inplace=True)
df_importances['importance'] = df_importances['importance'].map('{:.3}'.format)
df_importances

,feature,importance
14,Rolling_Avg_Rent_3_h,0.701
22,count_prev_week_same_hour,0.162
3,hour,0.0515
28,day_part_morning,0.0151
18,Rolling_Avg_Rent_12_h,0.00974
12,rush_hour,0.00763
13,casual_to_registered_ratio,0.00702
26,day_part_afternoon,0.00615
17,Rolling_Avg_Rent_6_h,0.00505
15,Rolling_Avg_Rent_4_h,0.00483


In [11]:
import numpy as np

# Make predictions
y_train_pred = rf_model.predict(X_train_scaled)
y_test_pred = rf_model.predict(X_test_scaled)

# Use regression metrics
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

print(f'Mean Squared Error on Training Set: {mse_train:.4f}')
print(f'Mean Squared Error on Test Set: {mse_test:.4f}')
print(f'Root Mean Squared Error on Test Set: {np.sqrt(mse_test):.4f}')

Mean Squared Error on Training Set: 101.3199
Mean Squared Error on Test Set: 723.5239
Root Mean Squared Error on Test Set: 26.8984


### Grid search

In [16]:
from sklearn.model_selection import GridSearchCV

grid_rf = {
    'n_estimators': [200, 500, 650],
    'max_depth': [10, 15, 25]
    #'min_samples_leaf': [1, 2, 4],
    #'min_samples_split': [2, 5, 10]
}

gs_rf = GridSearchCV(
    estimator=rf_model,
    param_grid= grid_rf,
    cv=3,
    verbose=2
)

gs_rf.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END .....................max_depth=10, n_estimators=100; total time=  11.5s
[CV] END .....................max_depth=10, n_estimators=100; total time=  11.7s
[CV] END .....................max_depth=10, n_estimators=100; total time=  11.9s
[CV] END .....................max_depth=10, n_estimators=150; total time=  18.4s
[CV] END .....................max_depth=10, n_estimators=150; total time=  18.1s
[CV] END .....................max_depth=10, n_estimators=150; total time=  19.6s
[CV] END .....................max_depth=10, n_estimators=200; total time=  27.2s
[CV] END .....................max_depth=10, n_estimators=200; total time=  28.2s
[CV] END .....................max_depth=10, n_estimators=200; total time=  27.1s
[CV] END .....................max_depth=15, n_estimators=100; total time=  21.4s
[CV] END .....................max_depth=15, n_estimators=100; total time=  19.2s
[CV] END .....................max_depth=15, n_est

GridSearchCV(cv=3,
             estimator=RandomForestRegressor(n_estimators=500, random_state=42),
             param_grid={'max_depth': [10, 15, 25],
                         'n_estimators': [100, 150, 200]},
             verbose=2)

In [17]:
best_params_rf = gs_rf.best_params_
best_rf = RandomForestRegressor(**best_params_rf)
best_rf.fit(X_train_scaled, y_train)
best_pred_rf = best_rf.predict(X_test_scaled)
print(best_params_rf)

{'max_depth': 25, 'n_estimators': 200}


In [18]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Make predictions
y_train_pred_gs = gs_rf.predict(X_train_scaled)
y_test_pred_gs = gs_rf.predict(X_test_scaled)

# Use regression metrics
mse_train = mean_squared_error(y_train, y_train_pred_gs)
mse_test = mean_squared_error(y_test, y_test_pred_gs)
r2_train = r2_score(y_train, y_train_pred_gs)
r2_test = r2_score(y_test, y_test_pred_gs)
mae_train = mean_absolute_error(y_train, y_train_pred_gs)
mae_test = mean_absolute_error(y_test, y_test_pred_gs)

print(f'Mean Squared Error on Training Set: {mse_train:.4f}')
print(f'Mean Squared Error on Test Set: {mse_test:.4f}')
print(f'R2 on Training Set: {r2_train:.4f}')
print(f'R2 on Test Set: {r2_test:.4f}')
print(f'MAE on Training Set: {mae_train:.4f}')
print(f'MAE on Test Set: {mae_test:.4f}')

Mean Squared Error on Training Set: 101.7968
Mean Squared Error on Test Set: 722.0598
R2 on Training Set: 0.9970
R2 on Test Set: 0.9771
MAE on Training Set: 5.9442
MAE on Test Set: 15.5709


In [1]:
!python -m streamlit run script_model_nano.py

^C


In [ ]:
# def train_time_series_with_folds(df, horizon=24*7):
#     X = df.drop('count', axis=1)
#     y = df['count']
    
#     #take last week of the dataset for validation
#     X_train, X_test = X.iloc[:-horizon,:], X.iloc[-horizon:,:]
#     y_train, y_test = y.iloc[:-horizon], y.iloc[-horizon:]
    
#     #create, train and do inference of the model
#     model = LGBMRegressor(random_state=42)
#     model.fit(X_train, y_train)
#     predictions = model.predict(X_test)
    
#     #calculate MAE
#     mae = np.round(mean_absolute_error(y_test, predictions), 3)    
    
#     #plot reality vs prediction for the last week of the dataset
#     fig = plt.figure(figsize=(16,8))
#     plt.title(f'Real vs Prediction - MAE {mae}', fontsize=20)
#     plt.plot(y_test, color='red')
#     plt.plot(pd.Series(predictions, index=y_test.index), color='green')
#     plt.xlabel('Hour', fontsize=16)
#     plt.ylabel('Number of Shared Bikes', fontsize=16)
#     plt.legend(labels=['Real', 'Prediction'], fontsize=16)
#     plt.grid()
#     plt.show()
    
#     #create a dataframe with the variable importances of the model
#     df_importances = pd.DataFrame({
#         'feature': model.feature_name_,
#         'importance': model.feature_importances_
#     }).sort_values(by='importance', ascending=False)
    
#     #plot variable importances of the model
#     plt.title('Variable Importances', fontsize=16)
#     sns.barplot(x=df_importances.importance, y=df_importances.feature, orient='h')
#     plt.show()